In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib as mpl

import math 

from bin import *

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [2]:
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': tex_preamble,
})

## Setup

In [3]:
n = 50
Λ = model_problem_spectrum(n,.8,1e3).astype(np.longdouble)
b = np.ones(n,dtype=np.longdouble)
b /= np.linalg.norm(b)

λmin = np.min(Λ).astype(np.double)
λmax = np.max(Λ).astype(np.double)
κ = λmax / λmin

In [4]:
w = 0

# norm to measure function approximation and linear system errors
B = Λ**2#np.ones(n)

A = np.diag(Λ)

In [5]:
K = 100
Q,(a_,b_) = lanczos_reorth(A.astype(np.single),b.astype(np.single),K,reorth=0)

Q,(a_,b_) = Q.astype(np.double),(a_.astype(np.double),b_.astype(np.double))

K_exact = 40
Q_exact,(a_exact,b_exact) = lanczos_reorth(A,b,K_exact,reorth=K_exact)

In [6]:
T = np.diag(a_) + np.diag(b_[:-1],1) + np.diag(b_[:-1],-1)
F_roundoff = A@Q[:,:-1] - Q[:,:-1]@T
F_roundoff[:,-1] -= b_[-1]*Q[:,-1]

In [7]:
plot_tol = 1e-20

In [8]:
T = np.zeros((3,K),dtype=np.double)
T[0,1:] = b_[:-1]
T[1] = a_-w
T[2,:-1] = b_[:-1]

e0 = np.zeros(K+1)
e0[0] = 1

In [9]:
err_CG = np.full(K,np.nan)

for k in range(0,K):
    try:
        e = (1/(Λ-w))*b - lanczos_FA(lambda x:1/(x-w),Q,a_,b_,k,normb=np.linalg.norm(b))
        err_CG[k] = np.sqrt( e.T*B@e )
    except: pass
    
    if err_CG[k] < plot_tol:
        break

In [10]:
res_CG = np.full(K,np.nan)

for k in range(0,K):
    try:
        r = b - A@lanczos_FA(lambda x:1/(x-w),Q,a_,b_,k,normb=np.linalg.norm(b))
        res_CG[k] = np.sqrt( r@r )
    except: pass
    
    if res_CG[k] < plot_tol:
        break

## $f(x) = 1/\sqrt{x}$, $\Gamma =$ circle

In [11]:
def f(x):
    return np.sqrt(x)

fAb = f(Λ)*b

In [12]:
err_lanczos = np.full(K,np.nan)

for k in range(0,K):
    try:
        e = fAb - lanczos_FA(f,Q.astype(np.double),a_,b_,k,normb=np.linalg.norm(b))
        err_lanczos[k] = np.sqrt( e.T*B@e )
    except: pass
    
    if err_lanczos[k] < plot_tol:
        break

In [13]:
err_lanczos_exact = np.full(K,np.nan)

for k in range(0,K_exact):
    try:
        e = fAb - lanczos_FA(f,Q_exact,a_exact,b_exact,k,normb=np.linalg.norm(b))
        err_lanczos_exact[k] = np.sqrt( e.T*B@e )
    except: pass
    
    if err_lanczos_exact[k] < plot_tol:
        break

In [14]:
def Gamma(t):
    z = np.exp((np.pi)*1j)*t
    dz = np.exp((np.pi)*1j)
    
    return z,dz

In [15]:
a_priori_bound_circle = np.full(K,np.inf)
a_posteriori_bound_circle = np.full(K,np.inf)

for k in range(1,K):
    a_priori_bound_circle[k-1] = 2*get_a_priori_bound(f,Gamma,[0,np.inf],k,w,λmin,λmax)[0]
    a_posteriori_bound_circle[k-1] = 2*get_a_posteriori_bound(f,Gamma,[0,np.inf],a_[:k],b_[:k-1],w,λmin,λmax)[0]

In [16]:
f_norms_full = np.full(K,np.nan)

for k in range(1,K):
    theta = sp.linalg.eigvalsh_tridiagonal(a_[:k],b_[:k-1],tol=1e-30)

    e0 = np.zeros(k,dtype='complex')
    e0[0] = 1
    
    T = np.zeros((3,k),dtype='complex')
    T[0,:-1] = b_[:k-1]
    T[1] = a_[:k] - w
    T[2,1:] = b_[:k-1]
    
    Twinve0 = sp.linalg.solve_banded((1, 1), T, e0)
    
    def F(t):

        z,dz = Gamma(t)
        
        T[1] = a_[:k] - z

        Tzinve0 = sp.linalg.solve_banded((1, 1), T, e0)

        fk = F_roundoff[:,:k]@ (Tzinve0 - np.prod((theta-w)/(theta-z)) * Twinve0)
        
        return (1/(2*np.pi)) * np.abs(f(z)) * Q_wz(w,z,λmin,λmax) * np.linalg.norm(fk) * np.abs(dz)
        
    integral = sp.integrate.quad(F,0,np.inf,epsabs=0,limit=200) 
    
    f_norms_full[k-1] = 2*integral[0]


<ipython-input-16-112006ce432a>:28: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  integral = sp.integrate.quad(F,0,np.inf,epsabs=0,limit=200)


In [17]:
fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))
ax = fig.add_axes([left, bottom, width, height])

ax.plot(a_posteriori_bound_circle*err_CG,**line_styles['l4'])
ax.plot(a_priori_bound_circle*err_CG,**line_styles['l3'])
ax.plot(err_lanczos_exact,**line_styles['l2'])
ax.plot(err_lanczos,**line_styles['l1'])

ax.set_xlabel('number of matvecs: $k$')
ax.set_yscale('log')

ax.set_ylim(1e-5,1e5)

ax.set_xlabel('number of matvecs: $k$')
ax.set_yscale('log')
ax.grid(True,linestyle=':',linewidth=.5)

#ax.legend()

plt.savefig('imgs/ch8_CIF.pdf')
plt.close()